<a href="https://colab.research.google.com/github/suinkangme/comp433_project/blob/main/COMP433_Project_DK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing a robust CNN model to address the challenge of learning with label noise in  CIFAR10 dataset

- CIFAR10 Label : ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’.

- image size : 3x32x32




In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, random_split
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [5]:
# Training on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load and normalize CIFAR10

In [ ]:
# transform_train = transforms.Compose([
#     transforms.RandomResizedCrop(32),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
# ])

# transform_val = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

# transform_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])



In [4]:
# Define the mean and standard deviation of CIFAR-10 dataset
cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std = (0.2023, 0.1994, 0.2010)

# Training transformations
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar10_mean, std=cifar10_std)
])

# Validation and testing transformations
transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar10_mean, std=cifar10_std)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar10_mean, std=cifar10_std)
])

In [6]:
# download dataset for training
cifar_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=None)

# 데이터를 train과 validation으로 나누기 위해 인덱스 생성
dataset_size = len(cifar_dataset)
validation_split = 0.2
val_size = int(validation_split * dataset_size)
train_size = dataset_size - val_size

# 데이터를 나누기
train_dataset, val_dataset = random_split(cifar_dataset, [train_size, val_size])

# 적절한 transform 적용
train_dataset.dataset.transform = transform_train
val_dataset.dataset.transform = transform_val

# 데이터 로더 설정
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


Files already downloaded and verified


In [7]:

# test dataset
test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                          train=False,
                                          download=True,
                                          transform = transform_test)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


## Define & train with a  baseline CNN model

In [8]:
class BaselineModel(nn.Module):
  def __init__(self):
    super(BaselineModel,self).__init__()
    self.features = nn.Sequential(
      nn.Conv2d(3, 8, kernel_size=3, padding = 1),  # (input channel, output channels, kernel size, padding)  32*32*8
      nn.ReLU(inplace=True), # activation function modifies the input tensor directly
      nn.Conv2d(8, 16, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=2,stride=2), # 16*16*16

      nn.Conv2d(16, 32, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.Conv2d(32, 64, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.Conv2d(64, 128, kernel_size=3, padding=1),
      nn.MaxPool2d(kernel_size=2,stride=2) # 8*8*128
    )

    # fully connected layers
    self.fc_layers = nn.Sequential(
      nn.Linear(128*8*8, 120),
      nn.ReLU(inplace=True),
      nn.Linear(120,84),
      nn.ReLU(inplace=True),
      nn.Linear(84,10)
    )


  def forward(self, x):
    x = self.features(x)
    x = torch.flatten(x,1)
    x = self.fc_layers(x)
    return x



In [9]:
base_model = BaselineModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(base_model.parameters(), lr=0.01, momentum=0.9)


In [10]:
base_model.train()
num_epochs = 10

for epoch in range(num_epochs):
    # Training
    base_model.train()
    total_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        # Move data to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = base_model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    average_train_loss = total_train_loss / len(train_loader)
    train_acc = 100 * correct_train / total_train

    # Validation
    base_model.eval()
    total_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            # Move data to GPU
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = base_model(inputs)
            loss = criterion(outputs, labels)

            total_val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    average_val_loss = total_val_loss / len(val_loader)
    val_acc = 100 * correct_val / total_val

    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Train Loss: {average_train_loss:.4f}, Train Accuracy: {train_acc:.2f}%, '
          f'Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {val_acc:.2f}')


Epoch 1/10, Train Loss: 1.8797, Train Accuracy: 30.47%, Validation Loss: 1.4473, Validation Accuracy: 46.93
Epoch 2/10, Train Loss: 1.3209, Train Accuracy: 52.56%, Validation Loss: 1.1206, Validation Accuracy: 59.38
Epoch 3/10, Train Loss: 1.0377, Train Accuracy: 63.13%, Validation Loss: 0.9842, Validation Accuracy: 65.30
Epoch 4/10, Train Loss: 0.8655, Train Accuracy: 69.50%, Validation Loss: 0.8609, Validation Accuracy: 69.64
Epoch 5/10, Train Loss: 0.7238, Train Accuracy: 74.56%, Validation Loss: 0.8665, Validation Accuracy: 70.44
Epoch 6/10, Train Loss: 0.6086, Train Accuracy: 78.30%, Validation Loss: 0.8313, Validation Accuracy: 72.25
Epoch 7/10, Train Loss: 0.5048, Train Accuracy: 82.10%, Validation Loss: 0.8742, Validation Accuracy: 71.56
Epoch 8/10, Train Loss: 0.4076, Train Accuracy: 85.45%, Validation Loss: 0.9367, Validation Accuracy: 71.90
Epoch 9/10, Train Loss: 0.3343, Train Accuracy: 88.10%, Validation Loss: 1.0634, Validation Accuracy: 70.11
Epoch 10/10, Train Loss: 0.2

In [11]:
# testing
base_model.eval()

# Variables to store predictions and ground truth labels
num_correct_predictions = 0
total_num_predictions = 0
test_loss = 0.0

# Iterate over the test dataset
with torch.no_grad():  # temporarily set all requires_grad flags to False
    for inputs, labels in test_loader:
        # Move data to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = base_model(inputs)
        loss = criterion(outputs, labels)

        # Compute loss and number of accurate predictions
        test_loss += loss.item()
        preds = outputs.argmax(dim=1)
        num_correct_predictions += (preds == labels).sum().item()
        total_num_predictions += labels.size(0)

# Compute average test loss
average_test_loss = test_loss / len(test_loader.dataset)

# Compute accuracy percentage
accuracy = (num_correct_predictions / total_num_predictions) * 100

print(f"Test Accuracy: {accuracy:.2f}%, Average Test Loss: {average_test_loss:.4f}")


Test Accuracy: 70.55%, Average Test Loss: 0.0182


## Noise Labeling
- 5 different noise levels (10%,
30%, 50%, 80%, 90%)

### Symmetric label noise

In [12]:
def replace_symmetric_noise(labels, epsilon):
    num_labels = len(labels)
    num_flips = int(epsilon * num_labels)

    # choose the label to be flipped
    flip_indices = np.random.choice(num_labels, num_flips, replace=True)

    # filp the label
    labels[flip_indices] = np.random.randint(0, 10, num_flips)

    return labels

### Asymmetric label noise

In [13]:
def flip_labels_asymmetrically(labels, epsilon):
    flip_rules = {
        9: 1,   # Truck to Automobile
        2: 0,   # Bird to Airplane
        4: 7,   # Deer to Horse
        3: 5,   # Cat to Dog
        5: 3,   # Dog to Cat
    }

    flipped_labels = []
    for label in labels:
        # Check if label flipping should occur based on epsilon
        if np.random.random() < epsilon:
            # Flip the label based on the flip_rules dictionary
            flipped_label = flip_rules.get(label, label)
            flipped_labels.append(flipped_label)
        else:
            # If no flipping, keep the original label
            flipped_labels.append(label)

    return np.array(flipped_labels)


## Train the model with the noise labeling

### Train with the symmetric noise labeling

In [23]:
# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]

# Create a dictionary with keys in the format 'noise_level_{100 * value}'
symmetric_model_dict = {f'noise_level_{int(100 * level)}_sy': None for level in noise_levels}

for epsilon in noise_levels:

    net = BaselineModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

    net = net.to(device)
    net.train()
    num_epochs = 7

    print(f"Symmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
        # Training
        net.train()
        total_train_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, labels in train_loader:

            labels_noisy = torch.from_numpy(replace_symmetric_noise(labels.numpy(), epsilon))

            # Move data to GPU
            inputs, labels_noisy = inputs.to(device), labels_noisy.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = net(inputs)
            loss = criterion(outputs, labels_noisy)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels_noisy.size(0)
            correct_train += (predicted == labels_noisy).sum().item()

        average_train_loss = total_train_loss / len(train_loader)
        train_acc = 100 * correct_train / total_train

        # Validation
        net.eval()
        total_val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                # Move data to GPU
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = net(inputs)
                loss = criterion(outputs, labels)

                total_val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        average_val_loss = total_val_loss / len(val_loader)
        val_acc = 100 * correct_val / total_val

        print(f'Epoch {epoch + 1}/{num_epochs}, '
              f'Train Loss: {average_train_loss:.4f}, Train Accuracy: {train_acc:.2f}%, '
              f'Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {val_acc:.2f}')

        # save model to dictionary
        symmetric_model_dict[f'noise_level_{int(100 * epsilon)}_sy'] = net


Symmetric Training with noise level: 0.1
Epoch 1/7, Train Loss: 2.0465, Train Accuracy: 24.59%, Validation Loss: 1.6988, Validation Accuracy: 39.54
Epoch 2/7, Train Loss: 1.6100, Train Accuracy: 44.62%, Validation Loss: 1.2724, Validation Accuracy: 54.08
Epoch 3/7, Train Loss: 1.4147, Train Accuracy: 53.60%, Validation Loss: 1.1544, Validation Accuracy: 60.34
Epoch 4/7, Train Loss: 1.2629, Train Accuracy: 60.12%, Validation Loss: 1.0042, Validation Accuracy: 65.99
Epoch 5/7, Train Loss: 1.1553, Train Accuracy: 64.68%, Validation Loss: 0.9017, Validation Accuracy: 69.29
Epoch 6/7, Train Loss: 1.0660, Train Accuracy: 68.82%, Validation Loss: 0.8539, Validation Accuracy: 71.17
Epoch 7/7, Train Loss: 0.9918, Train Accuracy: 72.17%, Validation Loss: 0.8692, Validation Accuracy: 71.36
Symmetric Training with noise level: 0.3
Epoch 1/7, Train Loss: 2.1463, Train Accuracy: 22.03%, Validation Loss: 1.7376, Validation Accuracy: 38.80
Epoch 2/7, Train Loss: 1.9215, Train Accuracy: 34.79%, Validat

### Testing

In [24]:
for key, net in symmetric_model_dict.items():
    net.eval()
    net.to(device)

    # Variables to store predictions and ground truth labels
    num_correct_predictions = 0
    total_num_predictions = 0
    loss = 0.0

    # Iterate over the test dataset
    with torch.no_grad():  # temporarily set all requires_grad flags to False
        for i, (data, label) in enumerate(test_loader):
            # move inputs to desired device and dtype
            data = data.to(device, dtype=torch.float32)
            label = label.to(device, dtype=torch.long)

            # forward pass
            logit = net(data)

            # compute loss and number of accurate predictions
            loss += torch.nn.functional.cross_entropy(logit, label, reduction='sum').item()
            preds = logit.max(dim=1)[1]
            num_correct_predictions += (preds == label).sum().item()
            total_num_predictions += len(preds)

        # compute average loss
        loss /= total_num_predictions

        # compute accuracy percentage
        accuracy = (float(num_correct_predictions) / total_num_predictions) * 100

        print(f"Symmetric {key} Accuracy: {accuracy:.2f}%, Loss: {loss:.4f}")


Symmetric noise_level_10_sy Accuracy: 72.27%, Loss: 0.8703
Symmetric noise_level_30_sy Accuracy: 67.47%, Loss: 1.1185
Symmetric noise_level_50_sy Accuracy: 63.19%, Loss: 1.3321
Symmetric noise_level_80_sy Accuracy: 56.61%, Loss: 1.5890
Symmetric noise_level_90_sy Accuracy: 56.72%, Loss: 1.6384


### Train with the asymmetric noise labeling

In [25]:
# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]
# Create a dictionary with keys in the format 'noise_level_{100 * value}'
asymmetric_model_dict = {f'noise_level_{int(100 * level)}_asy': None for level in noise_levels}



# Training on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epsilon in noise_levels:


    net = BaselineModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum = 0.9)


    net = net.to(device)
    net.train()
    num_epochs = 5


    print(f"Asymmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
      # Training
      net.train()
      total_train_loss = 0.0
      correct_train = 0
      total_train = 0

      for inputs, labels in train_loader:
          labels_noisy = torch.from_numpy(flip_labels_asymmetrically(labels.numpy(), epsilon))
          # Move data to GPU
          inputs, labels_noisy = inputs.to(device), labels_noisy.to(device)

          # Zero the gradients
          optimizer.zero_grad()

          # Forward pass
          outputs = net(inputs)
          loss = criterion(outputs, labels_noisy)

          # Backward pass and optimization
          loss.backward()
          optimizer.step()

          total_train_loss += loss.item()
          _, predicted = torch.max(outputs.data, 1)
          total_train += labels_noisy.size(0)
          correct_train += (predicted == labels_noisy).sum().item()

      average_train_loss = total_train_loss / len(train_loader)
      train_acc = 100 * correct_train / total_train

      # Validation
      net.eval()
      total_val_loss = 0.0
      correct_val = 0
      total_val = 0

      with torch.no_grad():
          for inputs, labels in val_loader:
              # Move data to GPU
              inputs, labels = inputs.to(device), labels.to(device)

              # Forward pass
              outputs = net(inputs)
              loss = criterion(outputs, labels)

              total_val_loss += loss.item()
              _, predicted = torch.max(outputs.data, 1)
              total_val += labels.size(0)
              correct_val += (predicted == labels).sum().item()

      average_val_loss = total_val_loss / len(val_loader)
      val_acc = 100 * correct_val / total_val

      print(f'Epoch {epoch + 1}/{num_epochs}, '
            f'Train Loss: {average_train_loss:.4f}, Train Accuracy: {train_acc:.2f}%, '
            f'Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {val_acc:.2f}')
    # save model to dictionary
    asymmetric_model_dict[f'noise_level_{int(100 * epsilon)}_asy'] = net


Asymmetric Training with noise level: 0.1
Epoch 1/5, Train Loss: 1.9821, Train Accuracy: 26.18%, Validation Loss: 1.7411, Validation Accuracy: 37.74
Epoch 2/5, Train Loss: 1.4303, Train Accuracy: 48.19%, Validation Loss: 1.2325, Validation Accuracy: 55.63
Epoch 3/5, Train Loss: 1.1628, Train Accuracy: 58.32%, Validation Loss: 1.1088, Validation Accuracy: 61.14
Epoch 4/5, Train Loss: 0.9861, Train Accuracy: 64.75%, Validation Loss: 0.9476, Validation Accuracy: 67.11
Epoch 5/5, Train Loss: 0.8570, Train Accuracy: 69.65%, Validation Loss: 0.8520, Validation Accuracy: 70.67
Asymmetric Training with noise level: 0.3
Epoch 1/5, Train Loss: 1.9450, Train Accuracy: 28.71%, Validation Loss: 1.7167, Validation Accuracy: 35.70
Epoch 2/5, Train Loss: 1.4159, Train Accuracy: 47.48%, Validation Loss: 1.2746, Validation Accuracy: 51.81
Epoch 3/5, Train Loss: 1.1787, Train Accuracy: 55.98%, Validation Loss: 1.1225, Validation Accuracy: 59.69
Epoch 4/5, Train Loss: 1.0312, Train Accuracy: 60.50%, Valid

### Testing

In [26]:
for key, net in asymmetric_model_dict.items():
    net.eval()
    net.to(device)

    # Variables to store predictions and ground truth labels
    num_correct_predictions = 0
    total_num_predictions = 0
    loss = 0.0

    # Iterate over the test dataset
    with torch.no_grad():  # temporarily set all requires_grad flags to False
        for i, (data, label) in enumerate(test_loader):
            # move inputs to desired device and dtype
            data = data.to(device, dtype=torch.float32)
            label = label.to(device, dtype=torch.long)

            # forward pass
            logit = net(data)

            # compute loss and number of accurate predictions
            loss += torch.nn.functional.cross_entropy(logit, label, reduction='sum').item()
            preds = logit.max(dim=1)[1]
            num_correct_predictions += (preds == label).sum().item()
            total_num_predictions += len(preds)

        # compute average loss
        loss /= total_num_predictions

        # compute accuracy percentage
        accuracy = (float(num_correct_predictions) / total_num_predictions) * 100

        print(f"Asymmetric {key} Accuracy: {accuracy:.2f}%, Loss: {loss:.4f}")

Asymmetric noise_level_10_asy Accuracy: 70.21%, Loss: 0.8703
Asymmetric noise_level_30_asy Accuracy: 68.22%, Loss: 0.9198
Asymmetric noise_level_50_asy Accuracy: 52.25%, Loss: 1.1821
Asymmetric noise_level_80_asy Accuracy: 45.68%, Loss: 1.3927
Asymmetric noise_level_90_asy Accuracy: 45.46%, Loss: 1.6320


# Active Passive Loss (APL)

**1. Active Loss Functions**:
<br> Cross Entropy (CE)
<br> Normalized Cross Entropy (NCE)
<br> Focal Loss (FL)
<br> Normalized Focal Loss (NFL)
<br>


<br> **2. Passive Loss Functions**:
<br> Mean Absolute Error (MAE)
<br> Normalized Mean Absolute Error (NMAE)
<br> Reverse Cross Entropy (RCE)
<br> Normalized Reverse Cross Entropy (NRCE)
<br>



In [49]:
class APLLoss(nn.Module):
    def __init__(self, alpha, beta):
        super(APLLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.active_loss = nn.CrossEntropyLoss()  # CE
        self.passive_loss = nn.L1Loss()  # MAE

    def forward(self, outputs, labels):
        active_loss = self.active_loss(outputs, labels)

        # label_one_hot = torch.nn.functional.one_hot(labels.clone().detach(), 10)
        # loss_passive = self.passive_loss(outputs, label_one_hot.view(-1, 1))
        # loss_passive = self.passive_loss(outputs, label_one_hot)
         # 수동적인 부분: 평균 절대 오차
        target_one_hot = F.one_hot(labels, num_classes=outputs.size(1)).float()
        passive_loss = F.l1_loss(F.softmax(outputs, dim=1), target_one_hot)

        apl_loss = (self.alpha * active_loss) + (self.beta * passive_loss)
        return apl_loss


In [56]:
alpha = 0.5  # 이 값은 조절 가능
beta = 0.5   # 이 값은 조절 가능
apl_criterion = APLLoss(alpha, beta)



In [57]:

APL_model = BaselineModel().to(device)
optimizer = optim.SGD(APL_model.parameters(), lr=0.01, momentum=0.9)

APL_model.train()
num_epochs = 5

for epoch in range(num_epochs):
    # Training
    APL_model.train()
    total_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        # Move data to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = APL_model(inputs)
        loss = apl_criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    average_train_loss = total_train_loss / len(train_loader)
    train_acc = 100 * correct_train / total_train

    # Validation
    APL_model.eval()
    total_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            # Move data to GPU
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = APL_model(inputs)
            loss = apl_criterion(outputs, labels)

            total_val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    average_val_loss = total_val_loss / len(val_loader)
    val_acc = 100 * correct_val / total_val

    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Train Loss: {average_train_loss:.4f}, Train Accuracy: {train_acc:.2f}%, '
          f'Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {val_acc:.2f}')


Epoch 1/5, Train Loss: 1.1233, Train Accuracy: 22.99%, Validation Loss: 0.9355, Validation Accuracy: 38.27
Epoch 2/5, Train Loss: 0.8480, Train Accuracy: 43.63%, Validation Loss: 0.7786, Validation Accuracy: 48.16
Epoch 3/5, Train Loss: 0.7153, Train Accuracy: 52.82%, Validation Loss: 0.7105, Validation Accuracy: 53.55
Epoch 4/5, Train Loss: 0.5988, Train Accuracy: 60.85%, Validation Loss: 0.5779, Validation Accuracy: 62.70
Epoch 5/5, Train Loss: 0.5089, Train Accuracy: 67.23%, Validation Loss: 0.5488, Validation Accuracy: 65.18


## Testing

In [52]:
# testing
APL_model.eval()

# Variables to store predictions and ground truth labels
num_correct_predictions = 0
total_num_predictions = 0
test_loss = 0.0

# Iterate over the test dataset
with torch.no_grad():  # temporarily set all requires_grad flags to False
    for inputs, labels in test_loader:
        # Move data to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = APL_model(inputs)
        loss = criterion(outputs, labels)

        # Compute loss and number of accurate predictions
        test_loss += loss.item()
        preds = outputs.argmax(dim=1)
        num_correct_predictions += (preds == labels).sum().item()
        total_num_predictions += labels.size(0)

# Compute average test loss
average_test_loss = test_loss / len(test_loader.dataset)

# Compute accuracy percentage
accuracy = (num_correct_predictions / total_num_predictions) * 100

print(f"Test Accuracy: {accuracy:.2f}%, Average Test Loss: {average_test_loss:.4f}")

Test Accuracy: 57.62%, Average Test Loss: 0.0187


## APL with noisy label 90

In [58]:
alpha = 0.6  # 이 값은 조절 가능
beta = 0.4   # 이 값은 조절 가능
apl_criterion = APLLoss(alpha, beta)

APL_model = BaselineModel().to(device)
optimizer = optim.SGD(APL_model.parameters(), lr=0.01, momentum=0.9)


In [59]:
APL_model.train()
num_epochs = 10

for epoch in range(num_epochs):
    # Training
    APL_model.train()
    total_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:

        labels_noisy = torch.tensor(flip_labels_asymmetrically(labels.cpu().numpy(), 0.9)).to(device)
        # Move data to GPU
        inputs, labels_noisy = inputs.to(device), labels_noisy.to(device)


        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = APL_model(inputs)
        loss = apl_criterion(outputs, labels_noisy)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels_noisy.size(0)
        correct_train += (predicted == labels_noisy).sum().item()

    average_train_loss = total_train_loss / len(train_loader)
    train_acc = 100 * correct_train / total_train

    # Validation
    APL_model.eval()
    total_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            # Move data to GPU
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = APL_model(inputs)
            loss = apl_criterion(outputs, labels)

            total_val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    average_val_loss = total_val_loss / len(val_loader)
    val_acc = 100 * correct_val / total_val

    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Train Loss: {average_train_loss:.4f}, Train Accuracy: {train_acc:.2f}%, '
          f'Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {val_acc:.2f}')


Epoch 1/10, Train Loss: 1.1213, Train Accuracy: 35.29%, Validation Loss: 1.3728, Validation Accuracy: 31.44
Epoch 2/10, Train Loss: 0.8426, Train Accuracy: 51.95%, Validation Loss: 1.2349, Validation Accuracy: 38.98
Epoch 3/10, Train Loss: 0.6976, Train Accuracy: 61.11%, Validation Loss: 1.1579, Validation Accuracy: 41.93
Epoch 4/10, Train Loss: 0.6113, Train Accuracy: 66.33%, Validation Loss: 1.1006, Validation Accuracy: 42.87
Epoch 5/10, Train Loss: 0.5426, Train Accuracy: 70.47%, Validation Loss: 1.1513, Validation Accuracy: 42.92
Epoch 6/10, Train Loss: 0.4883, Train Accuracy: 73.56%, Validation Loss: 1.0417, Validation Accuracy: 44.17
Epoch 7/10, Train Loss: 0.4408, Train Accuracy: 76.27%, Validation Loss: 1.1111, Validation Accuracy: 45.08
Epoch 8/10, Train Loss: 0.3961, Train Accuracy: 78.55%, Validation Loss: 0.9961, Validation Accuracy: 45.24
Epoch 9/10, Train Loss: 0.3418, Train Accuracy: 81.70%, Validation Loss: 1.0783, Validation Accuracy: 44.80
Epoch 10/10, Train Loss: 0.3

# Testing

In [39]:
# testing
APL_model.eval()

# Variables to store predictions and ground truth labels
num_correct_predictions = 0
total_num_predictions = 0
test_loss = 0.0

# Iterate over the test dataset
with torch.no_grad():  # temporarily set all requires_grad flags to False
    for inputs, labels in test_loader:
        # Move data to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = APL_model(inputs)
        loss = criterion(outputs, labels)

        # Compute loss and number of accurate predictions
        test_loss += loss.item()
        preds = outputs.argmax(dim=1)
        num_correct_predictions += (preds == labels).sum().item()
        total_num_predictions += labels.size(0)

# Compute average test loss
average_test_loss = test_loss / len(test_loader.dataset)

# Compute accuracy percentage
accuracy = (num_correct_predictions / total_num_predictions) * 100

print(f"Test Accuracy: {accuracy:.2f}%, Average Test Loss: {average_test_loss:.4f}")

Test Accuracy: 43.42%, Average Test Loss: 0.0285


# Different noises

In [63]:
def passive_loss_mae(outputs, targets):
    return F.l1_loss(outputs, targets)


In [64]:
def passive_loss_nmae(outputs, targets, epsilon=1e-8):
    # 절대 에러 계산
    absolute_errors = torch.abs(outputs - targets)

    # 절대 에러의 평균 계산
    absolute_mean = torch.mean(absolute_errors)

    # 타겟 범위 계산
    target_range = torch.max(targets) - torch.min(targets)

    # 정규화된 평균 절대 에러 계산
    return absolute_mean / (target_range + epsilon)

In [109]:
def passive_loss_nrce(outputs, targets, epsilon=1e-8):
    # 소프트맥스 함수를 사용하여 예측 확률 계산
    pred_prob = F.softmax(outputs, dim=1).to(outputs.device)

    # 정답의 원-핫 인코딩 생성
    true_prob = F.one_hot(targets, num_classes=outputs.size(1)).float().to(outputs.device)

    # 교차 엔트로피 계산
    cross_entropy = -torch.sum(true_prob * torch.log(pred_prob + epsilon), dim=1)

    # 예측의 엔트로피 계산
    entropy = -torch.sum(pred_prob * torch.log(pred_prob + epsilon), dim=1)

    # 정규화된 평균 교차 엔트로피 계산
    nrce = torch.mean(cross_entropy / entropy)

    return nrce

In [110]:
class APLLoss(nn.Module):
    def __init__(self, alpha, beta):
        super(APLLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.active_loss = nn.CrossEntropyLoss()  # CE

    def passive_loss(self, outputs, labels):
        return passive_loss_nrce(outputs, labels)

    def forward(self, outputs, labels):
        active_loss = self.active_loss(outputs, labels)
        passive_loss = self.passive_loss(outputs, labels)

        apl_loss = (self.alpha * active_loss) + (self.beta * passive_loss)
        return apl_loss

In [131]:
alpha = 0.2 # 이 값은 조절 가능
beta =  0.8   # 이 값은 조절 가능
apl_criterion = APLLoss(alpha, beta)

APL_model = BaselineModel().to(device)
optimizer = optim.SGD(APL_model.parameters(), lr=0.01, momentum=0.9)

In [132]:
APL_model.train()
num_epochs = 7

# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]
# Create a dictionary with keys in the format 'noise_level_{100 * value}'
asymmetric_model_dict = {f'noise_level_{int(100 * level)}_asy': None for level in noise_levels}



# Training on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epsilon in noise_levels:


    net = BaselineModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum = 0.9)


    net = net.to(device)
    net.train()


    print(f"Asymmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
      # Training
      net.train()
      total_train_loss = 0.0
      correct_train = 0
      total_train = 0

      for inputs, labels in train_loader:
          labels_noisy = torch.from_numpy(flip_labels_asymmetrically(labels.numpy(), epsilon))
          # Move data to GPU
          inputs, labels_noisy = inputs.to(device), labels_noisy.to(device)

          # Zero the gradients
          optimizer.zero_grad()

          # Forward pass
          outputs = net(inputs)
          loss = criterion(outputs, labels_noisy)

          # Backward pass and optimization
          loss.backward()
          optimizer.step()

          total_train_loss += loss.item()
          _, predicted = torch.max(outputs.data, 1)
          total_train += labels_noisy.size(0)
          correct_train += (predicted == labels_noisy).sum().item()

      average_train_loss = total_train_loss / len(train_loader)
      train_acc = 100 * correct_train / total_train

      # Validation
      net.eval()
      total_val_loss = 0.0
      correct_val = 0
      total_val = 0

      with torch.no_grad():
          for inputs, labels in val_loader:
              # Move data to GPU
              inputs, labels = inputs.to(device), labels.to(device)

              # Forward pass
              outputs = net(inputs)
              loss = criterion(outputs, labels)

              total_val_loss += loss.item()
              _, predicted = torch.max(outputs.data, 1)
              total_val += labels.size(0)
              correct_val += (predicted == labels).sum().item()

      average_val_loss = total_val_loss / len(val_loader)
      val_acc = 100 * correct_val / total_val

      print(f'Epoch {epoch + 1}/{num_epochs}, '
            f'Train Loss: {average_train_loss:.4f}, Train Accuracy: {train_acc:.2f}%, '
            f'Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {val_acc:.2f}')
    # save model to dictionary
    asymmetric_model_dict[f'noise_level_{int(100 * epsilon)}_asy'] = net

Asymmetric Training with noise level: 0.1
Epoch 1/7, Train Loss: 2.0170, Train Accuracy: 24.98%, Validation Loss: 1.6364, Validation Accuracy: 40.35
Epoch 2/7, Train Loss: 1.4791, Train Accuracy: 46.33%, Validation Loss: 1.2762, Validation Accuracy: 54.48
Epoch 3/7, Train Loss: 1.2203, Train Accuracy: 56.20%, Validation Loss: 1.1077, Validation Accuracy: 60.58
Epoch 4/7, Train Loss: 1.0352, Train Accuracy: 62.85%, Validation Loss: 0.9695, Validation Accuracy: 66.21
Epoch 5/7, Train Loss: 0.8953, Train Accuracy: 68.22%, Validation Loss: 0.8907, Validation Accuracy: 69.17
Epoch 6/7, Train Loss: 0.7768, Train Accuracy: 72.15%, Validation Loss: 0.8735, Validation Accuracy: 69.97
Epoch 7/7, Train Loss: 0.6722, Train Accuracy: 76.17%, Validation Loss: 0.8649, Validation Accuracy: 71.21
Asymmetric Training with noise level: 0.3
Epoch 1/7, Train Loss: 2.0277, Train Accuracy: 24.68%, Validation Loss: 1.6193, Validation Accuracy: 39.81
Epoch 2/7, Train Loss: 1.5138, Train Accuracy: 44.33%, Valid

In [133]:
for key, net in asymmetric_model_dict.items():
    net.eval()
    net.to(device)

    # Variables to store predictions and ground truth labels
    num_correct_predictions = 0
    total_num_predictions = 0
    loss = 0.0

    # Iterate over the test dataset
    with torch.no_grad():  # temporarily set all requires_grad flags to False
        for i, (data, label) in enumerate(test_loader):
            # move inputs to desired device and dtype
            data = data.to(device, dtype=torch.float32)
            label = label.to(device, dtype=torch.long)

            # forward pass
            logit = net(data)

            # compute loss and number of accurate predictions
            loss += torch.nn.functional.cross_entropy(logit, label, reduction='sum').item()
            preds = logit.max(dim=1)[1]
            num_correct_predictions += (preds == label).sum().item()
            total_num_predictions += len(preds)

        # compute average loss
        loss /= total_num_predictions

        # compute accuracy percentage
        accuracy = (float(num_correct_predictions) / total_num_predictions) * 100

        print(f"Asymmetric {key} Accuracy: {accuracy:.2f}%, Loss: {loss:.4f}")

Asymmetric noise_level_10_asy Accuracy: 70.57%, Loss: 0.8655
Asymmetric noise_level_30_asy Accuracy: 66.17%, Loss: 0.9777
Asymmetric noise_level_50_asy Accuracy: 56.64%, Loss: 1.0771
Asymmetric noise_level_80_asy Accuracy: 45.37%, Loss: 1.3952
Asymmetric noise_level_90_asy Accuracy: 43.99%, Loss: 1.7458
